## Establishing the environment

First of all, we must import the libraries we will use which, in our case, they are the `OpenAI` module from the `openai` library and the `os` library.

Let's recall that the OpenAI API key should be stored in a safe place and never hardcopy it explicitly in the code.

In [1]:
from openai import OpenAI
import os

# We load our OpenAI API (in my case, I have it saved
# as an environment variable to avoid copying it here explicitly)
openai_api_key = os.getenv("OPENAI_API_KEY")

# Initialize the OpenAI client
llm = OpenAI(api_key = openai_api_key)

# Let's build our chatbot

## Defining the expert role
Then, we define the kind of expert that our chatbot will be, defining its functionality through a message that we'll later pass on to the language model:

In [2]:
developer_prompt = {
'role': 'developer',
'content': """
    You are a friendly assistant and Python expert.
    Answer questions clearly and concisely.
    """
}

## The chatbot's heart

One of the most important parts is the function that takes care of calling the language model itself. By means of this function we can pass to the API and, therefore, to the model, which is what the user is asking for, how does he want to receive the answer, etc. Then, it calls the language model to process his questions and retrieve the result.

In [3]:
def get_response(dev_prompt, user_message):
    user_prompt = {
        "role": "user",
        "content": user_message
    }
    # API call
    response = llm.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            dev_prompt,
            user_prompt
        ],
    )

    # Returns the generated text
    return response.choices[0].message.content

## The main program

Once we defined the function that calls the LLM, we create the main program, which will count with a while loop so, after each answer, the program asks for the next question and so we can keep asking.

In [ ]:
print("Hello! I'm your Python assistant. Type 'exit' to end the conversation.")

while True:
    user_message = input("You: ")
    if user_message.lower() == "exit":
        print("Assistant: Goodbye!")
        break

    response = get_response(developer_prompt, user_message)
    print(f"Assistant: {response}")


# First tests

When we test the program we noticed that it also answers questions not related to Python.
Because of this, we must modify the `developer_prompt` to avoid it to answer things that are not related to its purpose. This is what it's called a *guardrail*.

In [5]:
developer_prompt = {
    "role": "developer",
    "content": """
        You are a friendly assistant and Python expert.
        Answer questions clearly and concisely.
        Only answer Python-related questions.
        Check if the request relates to any Python concept.
        If you receive a question that's not related to Python at all, say that you only know about Python, nothing else.
    """
}